# Imports

First, we'll import the necessary libraries.

In [4]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

Next, we'll query the City of Philadelphia data via the ArcGIS REST API using the `requests` library. We'll also use the `json` library to parse the response.

Finally, we'll use the `geopandas` library to create a geodataframe from the response.

We have three different datasets to import from the City's ArcGIS server. These are:

### 1. Vacant Land

In [52]:
# Define the URL for the Vacant_Indicators_Land feature service
land_url = 'https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/Vacant_Indicators_Land/FeatureServer/0/query'

# Define the parameters for the Vacant_Indicators_Land API request
land_params = {
    'where': '1=1',
    'outFields': '*',
    'returnGeometry': 'true',
    'f': 'json'
}

# Make the Vacant_Indicators_Land API request
land_response = requests.get(land_url, params=land_params)

# Check if the Vacant_Indicators_Land request was successful
if land_response.status_code == 200:
    # Convert the Vacant_Indicators_Land JSON data to a geopandas geodataframe; convert to CRS 2272
    land_data = land_response.json()

    # convert the JSON data to a pandas dataframe
    land_df = pd.DataFrame(land_data['features'])

    # separate the attributes column into one column per attribute
    land_df = pd.concat([land_df.drop(['attributes'], axis=1), land_df['attributes'].apply(pd.Series)], axis=1)

    # make the `geometry` column a shapely geometry object
    land_df['geometry'] = land_df['geometry'].apply(lambda x: Polygon(x['rings'][0]))

    # convert the pandas dataframe to a geopandas geodataframe
    land_gdf = gpd.GeoDataFrame(land_df, geometry='geometry', crs='EPSG:2272')

else:
    print('Vacant_Indicators_Land Request failed with status code:', land_response.status_code)

### 2. Vacant Buildings

In [53]:
# Define the URL for the Vacant_Indicators_Bldg feature service
bldg_url = 'https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/Vacant_Indicators_Bldg/FeatureServer/0/query'

# Define the parameters for the Vacant_Indicators_Bldg API request
bldg_params = {
    'where': '1=1',
    'outFields': '*',
    'returnGeometry': 'true',
    'f': 'json'
}

# Make the Vacant_Indicators_Bldg API request
bldg_response = requests.get(bldg_url, params=bldg_params)

# Check if the Vacant_Indicators_Bldg request was successful
if bldg_response.status_code == 200:
    # Convert the Vacant_Indicators_Bldg JSON data to a geopandas geodataframe
    bldg_data = bldg_response.json()

    # convert the JSON data to a pandas dataframe
    bldg_df = pd.DataFrame(bldg_data['features'])

    # separate the attributes column into one column per attribute
    bldg_df = pd.concat([bldg_df.drop(['attributes'], axis=1), bldg_df['attributes'].apply(pd.Series)], axis=1)

    # make the `geometry` column a shapely geometry object
    bldg_df['geometry'] = bldg_df['geometry'].apply(lambda x: Polygon(x['rings'][0]))

    # convert the pandas dataframe to a geopandas geodataframe
    bldg_gdf = gpd.GeoDataFrame(bldg_df, geometry='geometry', crs='EPSG:2272')

else:
    print('Vacant_Indicators_Bldg Request failed with status code:', bldg_response.status_code)

### 3. PHS Community Landcare Parcels

In [5]:
# Define the URL for the Vacant_Indicators_phs_landcare feature service
phs_landcare_url = 'https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/PHS_CommunityLandcare/FeatureServer/0/query'

# Define the parameters for the Vacant_Indicators_phs_landcare API request
phs_landcare_params = {
    'where': '1=1',
    'outFields': '*',
    'returnGeometry': 'true',
    'f': 'json'
}

# Make the Vacant_Indicators_phs_landcare API request
phs_landcare_response = requests.get(phs_landcare_url, params=phs_landcare_params)

# Check if the Vacant_Indicators_phs_landcare request was successful
if phs_landcare_response.status_code == 200:
    # Convert the Vacant_Indicators_phs_landcare JSON data to a geopandas geodataframe
    phs_landcare_data = phs_landcare_response.json()

    # convert the JSON data to a pandas dataframe
    phs_landcare_df = pd.DataFrame(phs_landcare_data['features'])

    # separate the attributes column into one column per attribute
    phs_landcare_df = pd.concat([phs_landcare_df.drop(['attributes'], axis=1), phs_landcare_df['attributes'].apply(pd.Series)], axis=1)

    # make the `geometry` column a shapely geometry object
    phs_landcare_df['geometry'] = phs_landcare_df['geometry'].apply(lambda x: Polygon(x['rings'][0]))

    # convert the pandas dataframe to a geopandas geodataframe
    phs_landcare_gdf = gpd.GeoDataFrame(phs_landcare_df, geometry='geometry', crs='EPSG:2272')

else:
    print('PHS_CommunityLandcare Request failed with status code:', phs_landcare_response.status_code)

Now we need to import several more datasets from the City's Carto database (SQL).

### 1. L&I Violations

In [20]:
import requests
import datetime

# Calculate one year ago from today's date
one_year_ago = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime("%Y-%m-%d")

# Modify the SQL query
li_sql_query = "SELECT service_request_id, subject, status, service_name, service_code, lat, lon FROM public_cases_fc WHERE requested_datetime >= '{}'".format(one_year_ago)

# Make the GET request
li_response = requests.get("https://phl.carto.com/api/v2/sql", params={"q": li_sql_query})

# Get the data
li_data = li_response.json()["rows"]

# convert li_data to a pandas dataframe
li_df = pd.DataFrame(li_data)


In [22]:
li_df.head()

,service_request_id,subject,status,service_name,service_code,the_geom,lat,lon
0,14801874,Graffiti Removal,Closed,Graffiti Removal,SR-CL01,0101000020E6100000DF8EAA05FAC952C08DD7598078FA...,39.956802,-75.155885
1,14807809,Graffiti Removal,Closed,Graffiti Removal,SR-CL01,0101000020E6100000F751156C06CA52C01A6A56AE1EF9...,39.946249,-75.156642
2,14808317,Graffiti Removal,Closed,Graffiti Removal,SR-CL01,0101000020E6100000AF106F002ACA52C0C81937F3B3FA...,39.958617,-75.158814
3,14933892,What day is trash/recycling collection in my n...,Closed,Information Request,SR-IR01,None,NaN,NaN
4,14933890,How do I recover my abandoned vehicle?,Closed,Information Request,SR-IR01,None,NaN,NaN


In [23]:
# Convert the data to a geopandas dataframe
li_gdf = gpd.GeoDataFrame(li_df, geometry=gpd.points_from_xy(li_df.lon, li_df.lat), crs='EPSG:2272')

# drop the lat and lon columns
li_gdf.drop(['lat', 'lon'], axis=1, inplace=True)

In [24]:
li_gdf.head()

,service_request_id,subject,status,service_name,service_code,the_geom,lat,lon,geometry
0,14801874,Graffiti Removal,Closed,Graffiti Removal,SR-CL01,0101000020E6100000DF8EAA05FAC952C08DD7598078FA...,39.956802,-75.155885,POINT (-75.156 39.957)
1,14807809,Graffiti Removal,Closed,Graffiti Removal,SR-CL01,0101000020E6100000F751156C06CA52C01A6A56AE1EF9...,39.946249,-75.156642,POINT (-75.157 39.946)
2,14808317,Graffiti Removal,Closed,Graffiti Removal,SR-CL01,0101000020E6100000AF106F002ACA52C0C81937F3B3FA...,39.958617,-75.158814,POINT (-75.159 39.959)
3,14933892,What day is trash/recycling collection in my n...,Closed,Information Request,SR-IR01,None,NaN,NaN,POINT EMPTY
4,14933890,How do I recover my abandoned vehicle?,Closed,Information Request,SR-IR01,None,NaN,NaN,POINT EMPTY
